<a href="https://colab.research.google.com/github/evansnjagi/spaceY/blob/main/SpaceX_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# getting the data from the spaceX api
# Objectives
- Booster version  from the rocket database
- Launch site and get, latitude, longitude, and the name
- Payload getting the mass kilograms and the orbit
- core(the outcome for landing). We will get the following data, type of landing, number of flights with the core, whether grids were used, whether the core we re-used, whether legs were used, landing pad used, block of the core( number used to separate version cores), number of times the specific core has been reused and finally serial of the core.

In [ ]:
%%capture
! pip install requests

In [ ]:
# library codes

import requests
import pandas as pd
import json


In [ ]:
# getting the entire dataset
response = requests.get("https://api.spacexdata.com/v4/launches/past")
data = pd.json_normalize(response.json())
print(type(data))
print(data.shape)
print(data.info())

In [ ]:
# booster version
rocket = data["rocket"]
BoosterVersion = []

for rocket in rocket:
    if rocket:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(rocket)).json()
        BoosterVersion.append(response["name"])
# boosterversion into a series

BoosterVersion = pd.Series(BoosterVersion)

In [ ]:
print(type(BoosterVersion))
print(f"The length of the booster version is: {len(BoosterVersion)}")

In [ ]:
from re import L
# launchpad, latitude, longitude, and the name of the lauchpad
launchpad = data["launchpad"]
LaunchSite = []
Latitude = []
Longitude = []

for launchpad in launchpad:
    if launchpad:
        response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(launchpad)).json()
        latitude = response["latitude"]
        longitude = response["longitude"]
        name = response["name"]
        LaunchSite.append(name)
        Latitude.append(latitude)
        Longitude.append(Longitude)
# into a series
LaunchSite = pd.Series(LaunchSite)
Latitude = pd.Series(Latitude)
Longitude = pd.Series(Longitude)

In [ ]:
print(type(LaunchSite))
print(f"The lenght of the launch site is: {len(LaunchSite)}")
print(type(Latitude))
print(f"The lenght of the Latitude is: {len(Latitude)}")
print(type(Longitude))
print(f"The length of the longitude is: {len(Longitude)}")

In [ ]:
# getting payload data, mass and orbit
Payloads = pd.Series(data["payloads"])
Payload = Payloads.apply(lambda x: x[0])

Mass_kg = []
Orbit = []

for payload in Payload:
    if payload:
        response = requests.get("https://api.spacexdata.com/v4/payloads/" + str(payload)).json()
        Mass_kg.append(response["mass_kg"])
        Orbit.append(response["orbit"])
Mass_kg = pd.Series(Mass_kg)
Orbit = pd.Series(Orbit)

In [ ]:
print(f"The length of the mass is: {len(Mass_kg)}")
print(type(Mass_kg))
print(f"The lenght of the orbit is: {len(Orbit)}")
print(type(Orbit))

In [ ]:
# Initialize lists
Block = []
ReusedCount = []
Serial = []
Outcome = []
Flights = []
GridFins = []
Legs = []
Reused = []
LandingPad = []

# Assuming 'data' is your DataFrame
cores = data["cores"]
Cores = cores.apply(lambda x: x[0] if len(x) > 0 else None)  # Handling empty lists

for core in Cores:
    if core:
        core_series = pd.Series(core)
        Core = core_series["core"]

        # Requesting the outcome of landing data
        response = requests.get("https://api.spacexdata.com/v4/cores/" + str(Core)).json()

        Block.append(response["block"])
        ReusedCount.append(response["reuse_count"])
        Serial.append("serial")

        Outcome.append(str(response.get('landing_success', 'None')) + ' ' + str(response.get('landing_type', 'None')))
        Flights.append(core_series.get('flight', None))
        GridFins.append(core_series.get('gridfins', None))
        Reused.append(core_series.get('reused', None))
        Legs.append(core_series.get('legs', None))
        LandingPad.append(core_series.get('landpad', None))
    else:
        Block.append(None)
        ReusedCount.append(None)
        Serial.append(None)
        Outcome.append(None)
        Flights.append(None)
        GridFins.append(None)
        Legs.append(None)
        Reused.append(None)
        LandingPad.append(None)

# Convert lists to series
Block = pd.Series(Block)
ReusedCount = pd.Series(ReusedCount)
Serial = pd.Series(Serial)
Outcome = pd.Series(Outcome)
Flights = pd.Series(Flights)
GridFins = pd.Series(GridFins)
Legs = pd.Series(Legs)
Reused = pd.Series(Reused)
LandingPad = pd.Series(LandingPad)


In [ ]:
print(f"The length of block is {len(Block)}")
print(type(Block))
print(f"The length of reused count is {len(ReusedCount)}")
print(type(ReusedCount))
print(f"The length of serial is {len(Serial)}")
print(type(Serial))
print(f"The length of outcome is {len(Outcome)}")
print(type(Outcome))
print(f"The length of flights is {len(Flights)}")
print(type(Flights))
print(f"The length of grid fins is {len(GridFins)}")
print(type(GridFins))
print(f"The length of legs is {len(Legs)}")
print(type(Legs))
print(f"The length of reused is {len(Reused)}")
print(type(Reused))
print(f"The length of landing pads is {len(LandingPad)}")
print(type(LandingPad))

In [ ]:
# creating a dictionary

launch_dict = {
    'FlightNumber': list(data['flight_number']),
    'Date': list(data['date_utc']),
    'BoosterVersion': BoosterVersion,
    'PayloadMass': Mass_kg,
    'LaunchSite': LaunchSite,
    'Orbit': Orbit,
    'Outcome': Outcome,
    'Flights': Flights,
    'GridFins': GridFins,
    "Reused": Reused,
    'ReusedCount': ReusedCount,
    'Block': Block,
    'Serial': Serial,
    'Legs': Legs,
    'LandingPad': LandingPad,
    'Latitude': Latitude,
    'Longitude': Longitude,
}

In [ ]:
# creating a dataframe
df = pd.DataFrame(launch_dict)
df.info()

In [ ]:
# getting total number of null values
df.isnull().sum()

In [ ]:
# filling the payload mass with the mean
mean = df["PayloadMass"].mean().round(2)
df["PayloadMass"] = df["PayloadMass"].fillna(mean)

In [ ]:
df.isnull().sum()

In [ ]:
data_falcon9 = df[df['BoosterVersion'] == 'Falcon 9']

In [ ]:
# reseting the flight number column
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9

In [ ]:
# saving the spaceX data
path = "spaceX_falcom9.csv"
pd.to_csv(path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# AUTHERED BY: EVANS NJAGI KARAGO